In [77]:
import pandas as pd
df = pd.read_csv('tahkeer_data_cleaned.csv', sep=',')
print(df.values)

[[0.00000000e+00 2.85714286e-01 1.42857143e-01 ... 0.00000000e+00
  6.11111111e-01 1.00000000e+00]
 [1.00000000e+00 3.57142857e-01 3.85714286e-01 ... 1.00000000e+00
  7.33333333e-01 0.00000000e+00]
 [2.00000000e+00 5.00000000e-01 2.14285714e-01 ... 0.00000000e+00
  4.22222222e-01 1.00000000e+00]
 ...
 [1.59253000e+05 1.42857143e-01 8.14285714e-01 ... 0.00000000e+00
  3.77777778e-01 0.00000000e+00]
 [1.59254000e+05 5.00000000e-01 3.57142857e-01 ... 0.00000000e+00
  4.55555556e-01 1.00000000e+00]
 [1.59255000e+05 7.14285714e-02 8.14285714e-01 ... 0.00000000e+00
  5.00000000e-01 0.00000000e+00]]


In [78]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

class BaggingClassifier:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth  
        self.models = [DecisionTreeClassifier(max_depth=self.max_depth) for _ in range(n_estimators)]  
    def fit(self, X, y):
        for model in self.models:
            indices = np.random.choice(len(X), len(X), replace=True)
            X_subset, y_subset = X.iloc[indices], y.iloc[indices] 
            model.fit(X_subset, y_subset)
    def predict(self, X, threshold=0.5):
        predictions = np.zeros((len(X), self.n_estimators))
        for i, model in enumerate(self.models):
            predictions[:, i] = model.predict(X)
        avg_predictions = np.mean(predictions, axis=1)
        binary_predictions = (avg_predictions >= threshold).astype(int)
        return binary_predictions

In [79]:
columns = df.columns.tolist()[1:]
print(columns)
columns.remove("smoking")
print(columns)
x=df[columns]
y=df["smoking"]

['weight(kg)', 'HDL', 'hearing(left)', 'triglyceride', 'fasting blood sugar', 'Cholesterol', 'LDL', 'AST', 'Gtp', 'eyesight(left)', 'waist(cm)', 'hearing(right)', 'serum creatinine', 'dental caries', 'systolic', 'smoking']
['weight(kg)', 'HDL', 'hearing(left)', 'triglyceride', 'fasting blood sugar', 'Cholesterol', 'LDL', 'AST', 'Gtp', 'eyesight(left)', 'waist(cm)', 'hearing(right)', 'serum creatinine', 'dental caries', 'systolic']


In [82]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.30,shuffle=False,train_size=0.70)
bagging_model = BaggingClassifier(n_estimators=100, max_depth=60)
bagging_model.fit(xtrain, ytrain)

In [83]:
from sklearn.metrics import accuracy_score
bagging_predictions = bagging_model.predict(xtest)
bagging_accuracy = accuracy_score(ytest, bagging_predictions)
print(f"Bagging Accuracy: {bagging_accuracy}")

Bagging Accuracy: 0.7417904696664811
